<a href="https://colab.research.google.com/github/Khalil-Ravikson/LSTM_For_Shak/blob/main/LSTM_For_Shak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop

In [ ]:
# Define a URL do arquivo
url = 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'

# Baixa o arquivo para um diretório temporário e retorna o caminho local
filepath = tf.keras.utils.get_file('shakespeare.txt', origin=url)

print(f'O arquivo está salvo em: {filepath}')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
O arquivo está salvo em: /root/.keras/datasets/shakespeare.txt


In [ ]:
# Abre o arquivo e lê o conteúdo para uma string
with open(filepath, 'r') as f:
    text = f.read()

# Imprime os primeiros 500 caracteres para verificar
print(f'Conteúdo do arquivo (primeiros 500 caracteres):\n{text[:500]}')

In [ ]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [ ]:
text

In [ ]:
# Seleciona os caracteres do índice 300.000 até o 800.000
subset_text = text[300000:800000]

In [ ]:
character = sorted(set(subset_text))
print(f"O vocabulário tem {len(character)} caracteres únicos.")

O vocabulário tem 39 caracteres únicos.


In [ ]:
char_to_index = {char: index for index, char in enumerate(character)}

print("Exemplos do dicionário char_to_index:")
print(char_to_index)

Exemplos do dicionário char_to_index:
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'a': 13, 'b': 14, 'c': 15, 'd': 16, 'e': 17, 'f': 18, 'g': 19, 'h': 20, 'i': 21, 'j': 22, 'k': 23, 'l': 24, 'm': 25, 'n': 26, 'o': 27, 'p': 28, 'q': 29, 'r': 30, 's': 31, 't': 32, 'u': 33, 'v': 34, 'w': 35, 'x': 36, 'y': 37, 'z': 38}


In [ ]:
index_to_char = {index: char for index, char in enumerate(character)}

print("Exemplos do dicionário index_to_char:")
print(index_to_char)

Exemplos do dicionário index_to_char:
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'a', 14: 'b', 15: 'c', 16: 'd', 17: 'e', 18: 'f', 19: 'g', 20: 'h', 21: 'i', 22: 'j', 23: 'k', 24: 'l', 25: 'm', 26: 'n', 27: 'o', 28: 'p', 29: 'q', 30: 'r', 31: 's', 32: 't', 33: 'u', 34: 'v', 35: 'w', 36: 'x', 37: 'y', 38: 'z'}


In [ ]:
sequence_length = 100
step_size = 3


In [ ]:
setence = []
next_char = []

for i in range(0, len(subset_text) - sequence_length, step_size):
    setence.append(subset_text[i:i+sequence_length])
    next_char.append(subset_text[i+sequence_length])

print(f'Número de sequências: {len(setence)}')
print(f'Exemplo de sequência de entrada: "{setence[0]}"')
print(f'Exemplo de caractere de saída: "{next_char[0]}"')

Número de sequências: 166634
Exemplo de sequência de entrada: " blunt,
and rice ap thomas with a valiant crew;
and many more of noble fame and worth:
and towards l"
Exemplo de caractere de saída: "o"


In [ ]:

setence_size = len(setence)
num_sequences = len(character)

# Inicializa as matrizes X e y com zeros
X = np.zeros((setence_size, sequence_length, num_sequences ), dtype=np.bool_)
y = np.zeros((setence_size,num_sequences), dtype=np.bool_)

# Preenche as matrizes com os valores one-hot encoding
for i, sentence in enumerate(setence):
    for t, character in enumerate(sentence):
        # Para X, a entrada one-hot é a sequência de caracteres
        X[i, t, char_to_index[character]] = 1
    # Para y, a saída one-hot é o caractere seguinte
    y[i, char_to_index[next_char[i]]] = 1

print(f'\nFormato de X: {X.shape}')
print(f'Formato de y: {y.shape}')


Formato de X: (166634, 100, 39)
Formato de y: (166634, 39)


In [ ]:
# Defina as dimensões de entrada com base nos seus dados
input_shape = (100, 39)  # (sequence_length, vocab_size)
vocab_size = 39

# 1. Crie o modelo sequencial
model = Sequential()

# 2. Adicione a camada LSTM
# A primeira camada LSTM precisa da dimensão de entrada (input_shape)
# 'return_sequences=True' é necessário se você for empilhar outra camada LSTM depois
model.add(LSTM(256, input_shape=input_shape))

# 3. Adicione a camada de saída (Dense)
# A camada de saída tem o mesmo tamanho do vocabulário (39)
# A função de ativação 'softmax' garante que a saída seja uma distribuição de probabilidade,
# onde a soma de todas as previsões é 1.
model.add(Dense(vocab_size, activation='softmax'))

# 4. Compile o modelo
# 'categorical_crossentropy' é a função de perda ideal para problemas de classificação multi-classe
# 'adam' é um otimizador popular e eficiente, mas testaremos 'RSMprop'
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))

# Resumo do modelo
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 256)            │       303,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 39)             │        10,023 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 313,127 (1.19 MB)

 Trainable params: 313,127 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X, y, batch_size=128, epochs=20)

Epoch 1/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 894s 685ms/step - loss: 2.6320
Epoch 2/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 884s 679ms/step - loss: 1.7883
Epoch 3/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 934s 688ms/step - loss: 1.6134
Epoch 4/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 910s 679ms/step - loss: 1.5193
Epoch 5/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 888s 682ms/step - loss: 1.4583
Epoch 6/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 919s 680ms/step - loss: 1.6383
Epoch 7/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 878s 674ms/step - loss: 2.2943
Epoch 8/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 885s 680ms/step - loss: 2.1657
Epoch 9/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 920s 679ms/step - loss: 2.3212
Epoch 10/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 878s 674ms/step - loss: 2.5027
Epoch 11/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 870s 668ms/step - loss: 2.2868
Epoch 12/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 941s 683ms/step - loss: 2.2288
Epoch 13/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 918s 680ms/step - loss: 2.1693
Epoch 14/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 891s

In [ ]:
model.save("shaksp.keras")

In [ ]:
def sample(preds, temperature=1.0):
    """
    Função de amostragem que escolhe um índice a partir de uma distribuição de probabilidade.

    Args:
        preds (np.ndarray): As probabilidades de previsão do modelo para cada caractere.
                            (ndarray de 1 dimensão)
        temperature (float): O fator de temperatura para ajustar a aleatoriedade da amostragem.

    Returns:
        int: O índice do caractere selecionado.
    """
    # Converter as previsões para float64 para evitar problemas de precisão numérica
    preds = np.asarray(preds).astype('float64') + 1e-10

    # Aplicar a temperatura para modificar as probabilidades
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)

    # Calcular as probabilidades após a temperatura
    preds = exp_preds / np.sum(exp_preds)

    # Gerar uma amostragem a partir da distribuição de probabilidade
    probas = np.random.multinomial(1, preds, 1)

    # Retornar o índice do caractere com a maior probabilidade na amostragem
    return np.argmax(probas)


In [ ]:
def generate_text(model, length, temperature, char_to_index, index_to_char, vocab_size, subset_text, sequence_length):
    """
    Gera texto a partir de uma semente aleatória.

    Args:
        model (keras.Model): O modelo LSTM treinado.
        length (int): O número de caracteres a serem gerados.
        temperature (float): O fator de temperatura para a amostragem.
        char_to_index (dict): Dicionário que mapeia caracteres para índices.
        index_to_char (dict): Dicionário que mapeia índices para caracteres.
        vocab_size (int): O tamanho do vocabulário.
        subset_text (str): O texto original de onde a semente será extraída.
        sequence_length (int): O comprimento da sequência de treinamento.

    Returns:
        str: O texto gerado.
    """
    # 1. Escolhe um índice de início aleatório para a semente
    start_index = random.randint(0, len(subset_text) - sequence_length - 1)

    # 2. Extrai a semente (texto inicial)
    generated_text = subset_text[start_index: start_index + sequence_length]

    # Imprime a semente para visualização
    print(f'Semente de entrada aleatória: "{generated_text}"')

    # 3. Loop de geração de texto
    for i in range(length):
        # 3.1. Pré-processa a string atual (generated_text) para o formato one-hot
        x_pred = np.zeros((1, sequence_length, vocab_size))
        for t, char in enumerate(generated_text):
            if char in char_to_index:
                x_pred[0, t, char_to_index[char]] = 1.0

        # 3.2. Faz a previsão com o modelo
        preds = model.predict(x_pred, verbose=0)[0]

        # 3.3. Amostra o próximo caractere com a função de temperatura
        next_index = sample(preds, temperature)
        next_char = index_to_char[next_index]

        # 3.4. Adiciona o caractere à string e move a janela
        generated_text += next_char
        generated_text = generated_text[1:]

    return generated_text

In [ ]:

print("Gerando texto com temperatura = 0.2 (mais conservador)...")
generated_low_temp = generate_text(
    model,
    length=500,
    temperature=0.2,
    char_to_index=char_to_index,
    index_to_char=index_to_char,
    vocab_size=vocab_size,
    subset_text=subset_text,
    sequence_length=100
)
print(generated_low_temp)

print("\nGerando texto com temperatura = 0.8 (mais criativo)...")
generated_high_temp = generate_text(
    model,
    length=500,
    temperature=0.8,
    char_to_index=char_to_index,
    index_to_char=index_to_char,
    vocab_size=vocab_size,
    subset_text=subset_text,
    sequence_length=100
)
print(generated_high_temp)

Gerando texto com temperatura = 0.2 (mais conservador)...
Semente de entrada aleatória: "ht;
and but thou love me, let them find me here:
my life were better ended by their hate,
than death"

and romeo: in the revem.

romeo:
here sor some starrive.

romeo:
no frear this sor.

romeo:
and so 

Gerando texto com temperatura = 0.8 (mais criativo)...
Semente de entrada aleatória: "cester:
now tell me, brother clarence, what think you
of this new marriage with the lady grey?
hath "
not my friends; for many and is all shall of an is ifmor!

king how sonone we
broes was richard to m


In [ ]:
print("\nGerando texto com temperatura = 0.9 (mais )...")
generated_high_temp = generate_text(
    model,
    length=500,
    temperature=0.9,
    char_to_index=char_to_index,
    index_to_char=index_to_char,
    vocab_size=vocab_size,
    subset_text=subset_text,
    sequence_length=100
)
print(generated_high_temp)


Gerando texto com temperatura = 0.9 (mais )...
Semente de entrada aleatória: "lows with thee for our day of doom.
this ague fit of fear is over-blown;
an easy task it is to win o"
 a brood and bine richire cland
to to man:
for wives srave
shoning now of banave.

nor--
aumer,
swan
